## OMDb API
This project uses OMDb API (http://www.omdbapi.com/) to search for movie information and store it to a MySQL database.

In [1]:
import requests
import json
import mysql.connector
import pandas as pd

#### Part 1
Search for all the movies containing the word "blade".

In [21]:
apikey = ''
title = 'blade'
searchtype = 'movie'
returntype = 'json'
API_version = '1'
plot = 'short'
url = 'http://www.omdbapi.com/'
headers = {'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36",
}

In [26]:
def searchByTitle(apikey, url, title):
    api_url = url + '?apikey=' + apikey + '&s=' + title + '&type=' + searchtype + '&v=' + API_version +'&r=' + returntype + '&plot=' + plot
    json_blade_movie = requests.request("POST", api_url, headers=headers)
    json_blade_movie = json_blade_movie.json()
    print("The URL string that would search for all the movies containing the word : "+ title + ": " + api_url)
    return json_blade_movie

In [25]:
searchByTitle(apikey, url, title)

The URL string that would search for all the movies containing the word : bladehttp://www.omdbapi.com/?apikey=ee58b5a0&s=blade&type=movie&v=1&r=json&plot=short


{'Search': [{'Title': 'Blade Runner',
   'Year': '1982',
   'imdbID': 'tt0083658',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BNzQzMzJhZTEtOWM4NS00MTdhLTg0YjgtMjM4MDRkZjUwZDBlXkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_SX300.jpg'},
  {'Title': 'Blade Runner 2049',
   'Year': '2017',
   'imdbID': 'tt1856101',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BNzA1Njg4NzYxOV5BMl5BanBnXkFtZTgwODk5NjU3MzI@._V1_SX300.jpg'},
  {'Title': 'Blade',
   'Year': '1998',
   'imdbID': 'tt0120611',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BOTk2NDNjZWQtMGY0Mi00YTY2LWE5MzctMGRhZmNlYzljYTg5XkEyXkFqcGdeQXVyMTAyNjg4NjE0._V1_SX300.jpg'},
  {'Title': 'Blade II',
   'Year': '2002',
   'imdbID': 'tt0187738',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BOWVjZTIzNDYtNTBlNC00NTJjLTkzOTEtOTE0MjlhYzI2YTcyXkEyXkFqcGdeQXVyNTAyODkwOQ@@._V1_SX300.jpg'},
  {'Title': 'Blade: Trinity',
   'Year': '2004',
   'imdbID': 't

#### Part 2
- Uses the OMDb API to search for information of my favourite movies.
- Connects to local SQL instance (MySQL), creates a database named "ucdavis" and a table "omdb" containing the columns: title, year, genre, director, imdb_rating, rotten_tomatoes, metacritic, plot, box_office.

In [38]:
topmovies = ['Titanic', '1900', 'Amelie', 'Trainspotting', 'Coherence', 'Inception']

In [39]:
def searchForID(apikey, url, title):
    api_url = url + '?apikey=' + apikey + '&t=' + title + '&type=' + searchtype + '&v=' + API_version +'&r=' + returntype + '&plot=' + plot
    response = requests.get(api_url) 
    response_json = response.json()
    return response_json['Title'], response_json['imdbID']

In [46]:
topmovies_imdbid = []
for topmovie in topmovies:
    title, imdbID = searchForID(apikey, url, topmovie)
    topmovies_imdbid.append([title, imdbID])
topmovies_imdbid = pd.DataFrame(topmovies_imdbid, columns = ["Title", "imdbID"])
topmovies_imdbid.head()

,Title,imdbID
0,Titanic,tt0120338
1,1900,tt0074084
2,Amelie,tt7686962
3,Trainspotting,tt0117951
4,Coherence,tt2866360


In [9]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="")
mycursor = mydb.cursor()
mycursor.execute("DROP DATABASE IF EXISTS ucdavis")
mycursor.execute("CREATE DATABASE ucdavis")
mycursor.execute("USE ucdavis")
mycursor.execute("DROP TABLE IF EXISTS omdb")
mycursor.execute("CREATE TABLE omdb(" 
                 "imdb_id VARCHAR(255),"
                 "title VARCHAR(255),"
                 "year VARCHAR(255),"
                 "director VARCHAR(255),"
                 "imdb_rating VARCHAR(10),"
                 "rotten_tomatoes VARCHAR(10),"
                 "metacritic VARCHAR(10),"
                 "plot TEXT(255) ,"
                 "box_office TEXT(255),"
                 "Genre VARCHAR(255))")

In [10]:
topmovies_information = []
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="happy100_",
  database = "ucdavis"
)
mycursor = mydb.cursor()
for imdbid in topmovies_imdbid['imdbID']:
    Rotten_Tomatoes = 'N/A'
    Metacritic_ = 'N/A'  
    OMDb_api_url_topmovie_information = 'http://www.omdbapi.com/?apikey='+apikey+'&i='+imdbid+'&type='+searchtype+'&v='+API_version+'&r='+returntype+'&plot=full'
    topmovie_information = requests.request("POST", OMDb_api_url_topmovie_information, headers=headers)
    topmovie_information = json.loads(topmovie_information.text)
    for i in range(len(topmovie_information['Ratings'])):
        if topmovie_information['Ratings'][i]['Source'] == 'Rotten Tomatoes':
            Rotten_Tomatoes = topmovie_information['Ratings'][1]['Value'][0:2]
        if topmovie_information['Ratings'][i]['Source'] == 'Metacritic':
            Metacritic = topmovie_information['Ratings'][2]['Value'][0:2]
    information = {'Year':topmovie_information['Year'],
                   'Title':topmovie_information['Title'], 
                   'imdbID':topmovie_information['imdbID'],
                   'Genre':topmovie_information['Genre'],
                   'Director':topmovie_information['Director'],
                   'imdbRating':topmovie_information['imdbRating'],
                   'Plot':topmovie_information['Plot'],
                   'BoxOffice':topmovie_information['BoxOffice'],
                   'Rotten Tomatoes':Rotten_Tomatoes,
                   'Metacritic':Metacritic
                  }
    topmovies_information.append(information)
    print(json.dumps(topmovie_information,indent = 4,sort_keys=True))
    mycursor.execute("INSERT INTO omdb"
                     "(imdb_id, title, year, director, imdb_rating, rotten_tomatoes, metacritic, plot, box_office,Genre)"
                     " VALUES (%(imdbID)s, %(Title)s, %(Year)s, %(Director)s, %(imdbRating)s, %(Rotten Tomatoes)s, "
                     "%(Metacritic)s, %(Plot)s, %(BoxOffice)s,%(Genre)s)",information)
    mydb.commit()
mydb.close()

{
    "Actors": "Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates",
    "Awards": "Won 11 Oscars. Another 114 wins & 83 nominations.",
    "BoxOffice": "N/A",
    "Country": "USA",
    "DVD": "10 Sep 2012",
    "Director": "James Cameron",
    "Genre": "Drama, Romance",
    "Language": "English, Swedish, Italian",
    "Metascore": "75",
    "Plot": "84 years later, a 100 year-old woman named Rose DeWitt Bukater tells the story to her granddaughter Lizzy Calvert, Brock Lovett, Lewis Bodine, Bobby Buell and Anatoly Mikailavich on the Keldysh about her life set in April 10th 1912, on a ship called Titanic when young Rose boards the departing ship with the upper-class passengers and her mother, Ruth DeWitt Bukater, and her fianc\u00e9, Caledon Hockley. Meanwhile, a drifter and artist named Jack Dawson and his best friend Fabrizio De Rossi win third-class tickets to the ship in a game. And she explains the whole story from departure until the death of Titanic on its first and last v

In [47]:
topmovies_information = pd.DataFrame(topmovies_information)
topmovies_information.head()

,BoxOffice,Director,Genre,Metacritic,Plot,Rotten Tomatoes,Title,Year,imdbID,imdbRating
0,N/A,James Cameron,"Drama, Romance",75,"84 years later, a 100 year-old woman named Ros...",89,Titanic,1997,tt0120338,7.8
1,N/A,Bernardo Bertolucci,"Drama, History",70,"Set in Italy, the film follows the lives and i...",47,1900,1976,tt0074084,7.7
2,N/A,"Aurore Damant, Aurore Deman",Short,70,N/A,N/A,Amelie,2017,tt7686962,N/A
3,N/A,Danny Boyle,Drama,83,"A wild, freeform, Rabelaisian trip through the...",90,Trainspotting,1996,tt0117951,8.1
4,N/A,James Ward Byrkit,"Drama, Horror, Mystery, Sci-Fi, Thriller",65,"On the night of an astronomical anomaly, eight...",88,Coherence,2013,tt2866360,7.2
